In [41]:
import pandas as pd
import re

In [42]:
df = pd.read_csv("/Users/admin/Desktop/repo/maison-pariès/etude_client_1191.csv", index_col=0)
df = df.rename(columns={'A quelle fréquence venez-vous nous rendre visite ?': 'frequence', 
                        'Depuis combien de temps êtes vous un client de la Maison Pariès ?': 'anciennete',
                        "Tranche d'âge" : "age",
                        "Quel est le produit que vous préférez ?" : "produit_prefere", 
                        "Quelle est votre catégorie socio-professionnelle ?" : "socio_pro",
                        "Comment nous avez-vous connus ?" : "conversion",
                        "de combien de personnes se compose votre foyer ?" : "foyer",
                        "Où achetez-vous nos produits ?" : 'lieu_achat',
                        "Si vous n'êtes pas de la région, aimeriez-vous pouvoir trouver nos produits à proximité de votre domicile ?" : "target_2_boutique_local"
                        })

In [43]:
df.columns

Index(['Horodateur', 'Boutique', 'code postal', 'anciennete', 'frequence',
       'conversion', 'En 2 mots, comment définiriez-vous la Maison ?',
       'A quelle(s) occasion(s) effectuez-vous des achats chez nous',
       'produit_prefere',
       'Quel est le dernier produit de la gamme Pariès que vous connaissez ?',
       '8. Quels sont les produits de nos gammes que vous achetez chez d’autres artisans ?',
       'Pourquoi ?', 'lieu_achat',
       'Avez-vous déjà eu l’occasion de commander nos produits sur notre site en ligne ?',
       'target_2_boutique_local', 'Pourquoi ?.1',
       'Que devons-nous faire pour progresser ?',
       '14. Dans les grandes évolutions de la consommation, quelles sont les 3 qui vous tiennent le plus à cœur ?',
       'Sexe', 'socio_pro', 'age', 'foyer',
       'S'il y a des enfants dans votre foyer, quel âge ont-ils ?',
       'Êtes-vous abonné à notre newsletter ?',
       'Si non, confiez-moi votre adresse mail pour recevoir notre actualité (moins 

In [44]:
def touriste_33(x):
    if re.search("^33", x):
        return "local"
    else:
        return "touriste"
    
def touriste_64(x):
    if re.search("^64", x):
        return "local"
    else:
        return "touriste"

df_bordeaux = df[df['Boutique'] == "Bordeaux"]
df_basque = df[df['Boutique'] != "Bordeaux"]
df_bordeaux["client"] = df['code postal'].apply(touriste_33)
df_basque["client"] = df['code postal'].apply(touriste_64)
df_recons = pd.concat([df_basque,df_bordeaux])
df = df_recons

/var/folders/rw/3l7m_pl90hgbgv97f87910n00000gn/T/ipykernel_1801/2803237770.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bordeaux["client"] = df['code postal'].apply(touriste_33)
/var/folders/rw/3l7m_pl90hgbgv97f87910n00000gn/T/ipykernel_1801/2803237770.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_basque["client"] = df['code postal'].apply(touriste_64)


In [45]:
def basque_choco_autre(x):
    if re.search("[Bb]asque", x):
        return "Gateau Basque"
    elif re.search("[Cc]hoco", x):
        return 'chocolat'
    else:
        return "autre"
    

df.dropna(subset=['produit_prefere'], inplace=True)
df['produit_prefere'].info()
df['target_produit_pref'] = df['produit_prefere'].apply(basque_choco_autre)
df['target_produit_pref'].value_counts()

<class 'pandas.core.series.Series'>
Int64Index: 1035 entries, 0 to 966
Series name: produit_prefere
Non-Null Count  Dtype 
--------------  ----- 
1035 non-null   object
dtypes: object(1)
memory usage: 16.2+ KB


Gateau Basque    496
autre            420
chocolat         119
Name: target_produit_pref, dtype: int64

In [46]:
fequence_converter = { "visite exceptionnelle" : "visite exceptionnel",
                      "visite mensuelle" :  "visite régulière",
                      "C'est la première fois !" : "première visite",
                      'visite hebdomadaire' : "visite régulière",
                      'visite exceptionnelle; Surtout en été' : 'visite exceptionnel',
                      'Tous les 15 jours' : "visite régulière",
                      '3 fois par semaine' : "visite régulière",
                      'Tous les jours' : "visite régulière", 
                      'Surtout en été' : "visite exceptionnel",
                      "Surtout l'été; Noël" : "visite exceptionnel",
                      'Surtout l’été' : "visite exceptionnel",
                      'visite exceptionnelle, Surtout en été' : "visite exceptionnel",
                      'Tous les 2 jours' : "visite régulière"
                      }

df = df.replace({"frequence": fequence_converter})
df['frequence'].unique()

array(['visite exceptionnel', 'visite régulière', 'première visite'],
      dtype=object)

In [47]:
def regex_codepostal(x):
    if x[0] in [str(i) for i in range(10)]:
        return x[0:2]
    else:
        return "étranger"

df["departement"] = df['code postal'].apply(regex_codepostal)
df["departement"]

0      31
1      64
2      62
3      33
4      64
       ..
961    64
962    33
964    33
965    33
966    49
Name: departement, Length: 1035, dtype: object

In [40]:
df.to_csv("dataset_prediction.csv")